In [25]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import *
import time

from fast_generator import fc_tree_commute_lookahead_fast
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from rustiq import pauli_network_synthesis, Metric
from rustiq.utils import entangling_count, entangling_depth

In [26]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, lookahead_size = 5):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            combined_time = end_time - start_time
            
            # Measure time for Qiskit method
            start_time = time.time()
            origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            qiskit_time = end_time - start_time

            # Measure time for RustiQ method
            start_time = time.time()
            circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            end_time = time.time()
            rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    "qiskit_time": qiskit_time,
                    "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    "rustiq_method": entangling_count(circuit)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "rustiq_method": entangling_depth(circuit)
                },
                "test_paulis_file": f'benchmarks/results/test_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [27]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, lookahead_size = 5, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "rustiq_method": entangling_depth(circuit)
        },
        "test_paulis_file": f'benchmarks/results/test_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [20]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,] #12]
orbitals_list = [4, 6, 8, 12, 16, 20,] #24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, lookahead_size = 5, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.1296851634979248, 'combined_time': 0.1605386734008789, 'qiskit_time': 0.15461993217468262, 'rustiq_time': 0.0009968280792236328}, 'gate_counts': {'our_method': 30, 'combined_method': 27, 'qiskit_method': 41, 'rustiq_method': 33}, 'circuit_entangling_depth': {'our_method': 24, 'combined_method': 22, 'qiskit_method': 41, 'rustiq_method': 31}, 'test_paulis_file': 'benchmarks/results/test_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 0.5245423316955566, 'combined_time': 0.7005312442779541, 'qiskit_time': 0.5822839736938477, 'rustiq_time': 0.006981372833251953}, 'gate_counts': {'our_method': 118, 'combined_method': 108, 'qiskit_method': 181, 'rustiq_method': 161}, 'circuit_entangling_depth': {'our_method': 96, 'combined_method': 86, 'qiskit_method': 178, 'rustiq_method': 145}, 'test_paulis_file': 'benchmarks/results/test_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 2.516925096511841, 'combined_time': 2.9948196411132812, 'qis

In [28]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, lookahead_size = 10)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 22.472973346710205, 'combined_time': 24.923808574676514, 'qiskit_time': 13.60697317123413, 'rustiq_time': 4.628634929656982}, 'gate_counts': {'our_method': 3556, 'combined_method': 3545, 'qiskit_method': 7602, 'rustiq_method': 3356}, 'circuit_entangling_depth': {'our_method': 2201, 'combined_method': 2192, 'qiskit_method': 7477, 'rustiq_method': 2321}, 'test_paulis_file': 'benchmarks/results/test_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 2.5097482204437256, 'combined_time': 2.8512701988220215, 'qiskit_time': 1.5299115180969238, 'rustiq_time': 0.02593064308166504}, 'gate_counts': {'our_method': 309, 'combined_method': 304, 'qiskit_method': 786, 'rustiq_method': 350}, 'circuit_entangling_depth': {'our_method': 214, 'combined_method': 211, 'qiskit_method': 769, 'rustiq_method': 267}, 'test_paulis_file': 'benchmarks/results/test_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.559504508972168, 'co

In [29]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, lookahead_size = 10)

max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 38.17032027244568, 'combined_time': 39.617900133132935, 'qiskit_time': 2.16821026802063, 'rustiq_time': 0.592900276184082}, 'gate_counts': {'our_method': 1017, 'combined_method': 1017, 'qiskit_method': 1798, 'rustiq_method': 1119}, 'circuit_entangling_depth': {'our_method': 571, 'combined_method': 571, 'qiskit_method': 412, 'rustiq_method': 681}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n10_e45_l20.json'}
max_cut_benchmark_n14_e60_l5.json
{'num_paulis': 370, 'times': {'our_time': 20.270930528640747, 'combined_time': 18.800795078277588, 'qiskit_time': 0.5245645046234131, 'rustiq_time': 0.0857703685760498}, 'gate_counts': {'our_method': 386, 'combined_method': 386, 'qiskit_method': 588, 'rustiq_method': 463}, 'circuit_entangling_depth': {'our_method': 195, 'combined_method': 195, 'qiskit_method': 200, 'rustiq_method': 275}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n14

In [30]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, lookahead_size = 10)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 3.651243209838867, 'combined_time': 3.777937173843384, 'qiskit_time': 0.13361048698425293, 'rustiq_time': 0.01795220375061035}, 'gate_counts': {'our_method': 104, 'combined_method': 104, 'qiskit_method': 296, 'rustiq_method': 116}, 'circuit_entangling_depth': {'our_method': 70, 'combined_method': 70, 'qiskit_method': 173, 'rustiq_method': 77}, 'test_paulis_file': 'benchmarks/results/test_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 12.060845136642456, 'combined_time': 12.693081140518188, 'qiskit_time': 0.5485358238220215, 'rustiq_time': 0.03690075874328613}, 'gate_counts': {'our_method': 308, 'combined_method': 308, 'qiskit_method': 896, 'rustiq_method': 337}, 'circuit_entangling_depth': {'our_method': 230, 'combined_method': 230, 'qiskit_method': 519, 'rustiq_method': 230}, 'test_paulis_file': 'benchmarks/results/test_labs_n10_layers3.json'}
labs_n11_layers1.json
{'num_paulis': 10